In [ ]:
%pip install yake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.0 MB/s eta 0:00:00


In [ ]:
%pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 776.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=a380662ca07fc951ebcbb5c74f8cf83d0b0b335985388a5400dc9f425fab223c
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from yake import KeywordExtractor
import pymorphy2
import re

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile

zip_file_path = '/content/drive/MyDrive/vk_hack/archive.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    with zip_ref.open('lenta-ru-news.csv') as file:
        df = pd.read_csv(file)

print(df.head())

                                           url  \
0   https://lenta.ru/news/1914/09/16/hungarnn/   
1  https://lenta.ru/news/1914/09/16/lermontov/   
2  https://lenta.ru/news/1914/09/17/nesteroff/   
3   https://lenta.ru/news/1914/09/17/bulldogn/   
4       https://lenta.ru/news/1914/09/18/zver/   

                                               title  \
0  1914. Русские войска вступили в пределы Венгрии     
1  1914. Празднование столетия М.Ю. Лермонтова от...   
2                           1914. Das ist Nesteroff!   
3                    1914. Бульдог-гонец под Льежем    
4           1914. Под Люблином пойман швабский зверь   

                                                text       topic  \
0  Бои у Сопоцкина и Друскеник закончились отступ...  Библиотека   
1  Министерство народного просвещения, в виду про...  Библиотека   
2  Штабс-капитан П. Н. Нестеров на днях, увидев в...  Библиотека   
3  Фотограф-корреспондент Daily Mirror рассказыва...  Библиотека   
4  Лица, приехавшие в 

<ipython-input-5-7cb714f09921>:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [ ]:
df.drop(columns=['url'], inplace=True)
df.dropna(subset=['text'], inplace=True)
df.shape

(800970, 5)

In [ ]:
def work(text : str):
    morph = pymorphy2.MorphAnalyzer()
    russian_pattern = re.compile('[а-яё]+', re.I)

    extractor2 = KeywordExtractor(
        lan = "ru",
        n = 2,
        dedupLim = 0.6 ,
        top = 5
    )

    extractor1 = KeywordExtractor(
        lan = "ru",
        n = 1,
        dedupLim = 0.6,
        top = 5
    )

    def is_russian_word(word):
        russian_pattern = re.compile('[а-яё]+', re.I)
        return bool(russian_pattern.match(word))


    def normalize_tag(tag):
        parsed_tag = morph.parse(tag)[0]
        normalized_tag = parsed_tag.normal_form
        return normalized_tag



    k2 = extractor2.extract_keywords(text)
    k1 = extractor1.extract_keywords(text)

    keywords = k2[:1]
    keywords.extend(k1)
    keywords.extend(k1[1:])

    filtered_keywords = []

    for kw, score in keywords:
        nkw = normalize_tag(kw)
        kw = list(kw.split())

        f = 0
        s = []
        for i in range(len(kw)):
            if not is_russian_word(kw[i]):
                s.append(kw[i])
                continue
            if 'новость' in nkw or ' риа' in nkw or nkw == 'риа':
                f = 1
                break
            parsed_word = morph.parse(kw[i])[0]
            if parsed_word.inflect({'nomn'}):
                singular_nominative = parsed_word.inflect({'nomn'}).word
                s.append(singular_nominative.title())

        if not f and len(s) > 0:
          filtered_keywords.append(' '.join(s))

    fk = list(set(filtered_keywords[:4]))
    return {'keywords': fk}

In [ ]:
news = df.sample(10)
news['keywords'] = news['text'].apply(work)

In [ ]:
news

,title,text,topic,tags,date,keywords
466128,Джордж Майкл выпал из машины на полном ходу,Британский певец Джордж Майкл вывалился из авт...,Из жизни,Происшествия,2013/05/21,"{'keywords': ['Скорость', 'Британский', 'Майкл..."
537989,Прокуратура объявила Кернеса подозреваемым,Генеральная прокуратура Украины официально объ...,Бывший СССР,Украина,2015/03/04,"{'keywords': ['Украина', 'Харьков Геннадий', '..."
253655,Жители Калифорнии проголосовали за запрет одно...,Жители Калифорнии поддержали запрет на однопол...,Мир,Все,2008/11/06,"{'keywords': ['Запрет', 'Калифорния', 'Браки']}"
571936,Турецкий премьер отверг возможность военной оп...,Анкара не собирается проводить военную операци...,Мир,Политика,2015/12/05,"{'keywords': ['Исламское', 'Исламское Государс..."
548329,Саакашвили получил гражданство Украины,Бывший президент Грузии Михаил Саакашвили полу...,Бывший СССР,Украина,2015/05/30,"{'keywords': ['Украина', 'Саакашвили', 'Михаил..."
476362,Dragon's Crown выйдет в Европе в октябре,Стала известна дата европейского релиза игры D...,Наука и техника,Игры,2013/08/15,"{'keywords': ['Dragon', 'Dragon Crown', 'Экшн'..."
242026,ООН раскритиковала Великобританию за ограничен...,Комитет ООН по правам человека в своем новом д...,Мир,Все,2008/08/15,"{'keywords': ['Оон', 'The Guardian', 'Великобр..."
92447,Клинтону прислали письмо с белым порошком,Офис бывшего президента США Билла Клинтона (Bi...,Россия,Все,2004/04/28,"{'keywords': ['Bill', 'Clinton', 'Bill Clinton..."
583026,SpaceX отменила старт ракеты Falcon 9 за две м...,Компания SpaceX отменила запуск ракеты Falcon ...,Наука и техника,Космос,2016/02/26,"{'keywords': ['Ракеты', 'Falcon', 'Компания', ..."
246033,Банк Lehman Brothers лишился потенциальных пок...,Британский банк Barclays и американский Bank o...,Экономика,Все,2008/09/15,"{'keywords': ['Lehman', 'Brothers', 'Barclays'..."


In [ ]:
!pip freeze > requirements.txt

In [ ]:
news.iloc[3].text

'В Башкирии в городе Кумертау после тушения пожара обнаружены тела пяти человек с ножевыми ранениями. Об этом сообщает сайт Следственного управления Следственного комитета по республике. Следователи установили личности убитых, среди которых были двое несовершеннолетних. «Ими оказались 16-летняя хозяйка квартиры, 17-летняя подруга, их общая 21-летняя подруга со своим 22-летним сожителем и женщина 21 года. Все жители города Кумертау»,\xa0— уточнили в управлении. В настоящее время продолжается осмотр места происшествия. Следователи проверяют версию о совершении преступления лицом или лицами из круга общения погибших. Тела погибших направлены на судебно-медицинскую экспертизу. Как сообщил телеканалу LifeNews источник в правоохранительных органах, в двухкомнатной квартире, где были убиты пять человек, не обнаружено следов алкоголя или наркотиков. Соседи не слышали шума. Дверь была заперта. По факту произошедшего возбуждено уголовное дело по пункту «а» части\xa02 статьи\xa0105 («Убийство дву